In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/labeled_training (2).csv')
df_dev = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/SMM4H_2024_Task6_Dev.csv')
df_test = pd.read_csv('/content/drive/MyDrive/SMM4H 2024/unlabeled_testing.csv')

In [3]:
print(len(df_train))
print(len(df_dev))
print(len(df_test))

8800
4200
8200


In [4]:
import re
import pandas as pd
def preprocess_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.lower()
    text = re.sub(r'[@]\w+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df_train['text'] = df_train['text'].apply(preprocess_text)
df_dev['text'] = df_dev['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [5]:
!pip install transformers seqeval torch tqdm accelerate sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvid

In [6]:
print(df_train['source'].value_counts())
print(df_dev['source'].value_counts())
print(df_test['source'].value_counts())

source
twitter    8800
Name: count, dtype: int64
source
twitter    2200
reddit     2000
Name: count, dtype: int64
source
reddit     6000
twitter    2200
Name: count, dtype: int64


In [7]:
df_train_reddit = df_dev[df_dev['source'] == 'reddit']
df_train_twitter = pd.concat([df_train, df_dev[df_dev['source'] == 'twitter']])
df_test_reddit = df_test[df_test['source'] == 'reddit']
df_test_twitter = df_test[df_test['source'] == 'twitter']

In [8]:
print(len(df_train_reddit))
print(len(df_train_twitter))
print(len(df_test_reddit))
print(len(df_test_twitter))

2000
11000
6000
2200


In [9]:
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy, TrainerCallback
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from accelerate import Accelerator
import numpy as np
import torch
import os
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
class SaveModelCallback(TrainerCallback):
    def __init__(self, output_dir):
        self.output_dir = output_dir
    def on_epoch_end(self, args, state, control, **kwargs):
        output_dir = os.path.join(self.output_dir, f"epoch_bartlarge_B_{state.epoch}")
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
texts = df_train_twitter['text'].tolist()
labels = df_train_twitter['label'].tolist()
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=2) # Adjust num_labels as per your requirement
train_dataset = TextDataset(tokenizer, texts, labels)
training_args = TrainingArguments(
    output_dir='./results_temp',
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=0,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy=IntervalStrategy.NO,
    save_strategy=IntervalStrategy.NO,
    load_best_model_at_end=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[SaveModelCallback(output_dir='./model_saves')]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,0.486400
200,0.369700
300,0.350400
400,0.360700
500,0.308900
600,0.320700
700,0.319500
800,0.297400
900,0.292600
1000,0.290400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=27500, training_loss=0.06366531334060922, metrics={'train_runtime': 7834.8418, 'train_samples_per_second': 28.08, 'train_steps_per_second': 3.51, 'total_flos': 6.350887980096e+16, 'train_loss': 0.06366531334060922, 'epoch': 20.0})

In [10]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels if labels is not None else [0] * len(texts)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

In [11]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_1.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_1 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_1.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_1 = pd.concat([df_test_twitter, df_predictions_1], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_1.to_csv('BartLargeA_task6_predictions_069_epoch_1.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_2.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_2 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_2.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_2 = pd.concat([df_test_twitter, df_predictions_2], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_2.to_csv('BartLargeA_task6_predictions_069_epoch_2.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_3.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_3 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_3.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_3 = pd.concat([df_test_twitter, df_predictions_3], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_3.to_csv('BartLargeA_task6_predictions_069_epoch_3.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_4.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_4 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_4.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_4 = pd.concat([df_test_twitter, df_predictions_4], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_4.to_csv('BartLargeA_task6_predictions_069_epoch_4.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_5.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_5 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_5.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_5 = pd.concat([df_test_twitter, df_predictions_5], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_5.to_csv('BartLargeA_task6_predictions_069_epoch_5.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_6.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_6 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_6.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_6 = pd.concat([df_test_twitter, df_predictions_6], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_6.to_csv('BartLargeA_task6_predictions_069_epoch_6.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_7.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_7 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_7.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_7 = pd.concat([df_test_twitter, df_predictions_7], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_7.to_csv('BartLargeA_task6_predictions_069_epoch_7.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_8.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_8 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_8.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_8 = pd.concat([df_test_twitter, df_predictions_8], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_8.to_csv('BartLargeA_task6_predictions_069_epoch_8.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [19]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_9.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_9 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_9.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_9 = pd.concat([df_test_twitter, df_predictions_9], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_9.to_csv('BartLargeA_task6_predictions_069_epoch_9.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_10.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_10 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_10.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_10 = pd.concat([df_test_twitter, df_predictions_10], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_10.to_csv('BartLargeA_task6_predictions_069_epoch_10.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_11.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_11 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_11.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_11 = pd.concat([df_test_twitter, df_predictions_11], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_11.to_csv('BartLargeA_task6_predictions_069_epoch_11.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [22]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_12.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_12 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_12.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_12 = pd.concat([df_test_twitter, df_predictions_12], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_12.to_csv('BartLargeA_task6_predictions_069_epoch_12.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [23]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_13.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_13 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_13.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_13 = pd.concat([df_test_twitter, df_predictions_13], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_13.to_csv('BartLargeA_task6_predictions_069_epoch_13.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [24]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_14.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_14 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_14.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_14 = pd.concat([df_test_twitter, df_predictions_14], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_14.to_csv('BartLargeA_task6_predictions_069_epoch_14.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [25]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_15.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_15 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_15.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_15 = pd.concat([df_test_twitter, df_predictions_15], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_15.to_csv('BartLargeA_task6_predictions_069_epoch_15.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [26]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_16.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_16 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_16.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_16 = pd.concat([df_test_twitter, df_predictions_16], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_16.to_csv('BartLargeA_task6_predictions_069_epoch_16.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [27]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_17.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_17 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_17.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_17 = pd.concat([df_test_twitter, df_predictions_17], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_17.to_csv('BartLargeA_task6_predictions_069_epoch_17.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_18.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_18 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_18.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_18 = pd.concat([df_test_twitter, df_predictions_18], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_18.to_csv('BartLargeA_task6_predictions_069_epoch_18.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [29]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_19.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_19 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_19.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_19 = pd.concat([df_test_twitter, df_predictions_19], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_19.to_csv('BartLargeA_task6_predictions_069_epoch_19.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [30]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSequenceClassification.from_pretrained('/content/model_saves/epoch_bartlarge_B_20.0')                       #  A/B 1.0/20.0
model = model.to('cuda')
dev_texts = df_test_twitter['text'].tolist()                                                                                    # reddit/twitter
dev_dataset = TextDataset(tokenizer, dev_texts)
dev_loader = DataLoader(dev_dataset, batch_size=4, shuffle=False)
predictions = []
for batch in tqdm(dev_loader, desc="Evaluating"):
    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    pred_labels = logits.argmax(1)
    predictions.extend(zip(pred_labels.cpu().numpy(), probabilities[:,0].cpu().numpy(), probabilities[:,1].cpu().numpy()))
df_predictions_20 = pd.DataFrame(predictions, columns=['predicted_label', 'prediction_0_weight', 'prediction_1_weight'])       #_1.0/_20.0
df_test_twitter.reset_index(drop=True, inplace=True)                                                                           # reddit/twitter
df_predictions_20.reset_index(drop=True, inplace=True)                                                                         #_1.0/_20.0
df_merged_20 = pd.concat([df_test_twitter, df_predictions_20], axis=1)                                                           # reddit/twitter #_1.0/_20.0 #_1.0/_20.0
df_merged_20.to_csv('BartLargeA_task6_predictions_069_epoch_20.csv', index=False)                                               # A/B            #_1.0/_20.0 #_1/_20

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


Evaluating:   0%|          | 0/550 [00:00<?, ?it/s]

In [46]:
df_merged_20['predicted_label'].value_counts()

predicted_label
0    1448
1     752
Name: count, dtype: int64

In [50]:
df_merged_20

,id,text,source,predicted_label,prediction_0_weight,prediction_1_weight
0,15001,my 21st birthday is july 10th and im more than...,twitter,1,0.000003,0.999997
1,15002,youre 42 years old rt : is it might as well or...,twitter,0,0.999998,0.000002
2,15003,poor george given up to make room for baby. le...,twitter,0,0.999999,0.000001
3,15004,2/3 of the legislature under the age of 55 are...,twitter,0,0.999999,0.000001
4,15005,37th bday!,twitter,1,0.000026,0.999974
...,...,...,...,...,...,...
2195,17196,a little 9 year old girl who look laka likkle ...,twitter,0,0.999999,0.000001
2196,17197,bus naam he kafi hai no one can look so fit at...,twitter,0,0.999999,0.000001
2197,17198,yeah going to the tanning bed is cute i guess ...,twitter,0,0.999998,0.000002
2198,17199,my fucking 21st birthday shit was to live and ...,twitter,0,0.999988,0.000012


In [51]:
submission_part_B = df_merged_20[['id','source','predicted_label']]
submission_part_B = submission_part_B.rename(columns={'predicted_label': 'label'})
submission_part_B.to_csv('submission_part_B.csv', index=False)

In [52]:
submission_part_B

,id,source,label
0,15001,twitter,1
1,15002,twitter,0
2,15003,twitter,0
3,15004,twitter,0
4,15005,twitter,1
...,...,...,...
2195,17196,twitter,0
2196,17197,twitter,0
2197,17198,twitter,0
2198,17199,twitter,0
